In [ ]:
import data_loader as dl
import market_data_loader as mdl

file_path_asset = r'data\asset_data.xlsx'
file_path_portfolio = r'data\portfolio_data.xlsx'

asset_data = dl.convert_to_csv(file_path_asset)
portfolio_data = dl.convert_to_csv(file_path_portfolio)

market_data = mdl.load_market_data(asset_data)

asset_market_data = market_data[0]
fx_market_data = market_data[1]

price_today = asset_market_data.iloc[0]
fx_today = fx_market_data.iloc[0]


[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  2 of 2 completed

Ticker      EURCHF=X  USDCHF=X  CHFCHF=X
Date                                    
2025-01-09   0.93957   0.91095       1.0
2025-01-10   0.93915   0.91207       1.0
2025-01-13   0.93912   0.91677       1.0
2025-01-14   0.93998   0.91578       1.0
2025-01-15   0.94039   0.91246       1.0
...              ...       ...       ...
2026-01-06   0.92792   0.79210       1.0
2026-01-07   0.92997   0.79550       1.0
2026-01-08   0.93140   0.79765       1.0
2026-01-09   0.93160   0.79901       1.0
2026-01-10   0.93160   0.80090       1.0

[259 rows x 3 columns]
